# Accessible Gaussian Splatting — Evaluation on Google Colab (T4)

This notebook runs a non-interactive evaluation of the Accessible Gaussian Splatting pipeline on Tanks and Temples and Deep Blending datasets using a single 7k iteration point.

- Datasets are assumed to be stored in Google Drive.
- The notebook clones the repo, installs dependencies, runs training + rendering + metrics via `full_eval.py`.
- A summary of metrics is produced and results are copied back to Drive for later comparison with baselines from the original paper.

# Setup

In [ ]:
# Mount Google Drive and set dataset paths (update these if needed)
from google.colab import drive
drive.mount('/content/drive')

# Root folders in Drive containing the datasets
TAT_DIR = '/content/drive/MyDrive/tandt_db/tandt'
DB_DIR  = '/content/drive/MyDrive/tandt_db/db'

# Local output directory in Colab
OUTPUT_DIR = '/content/accesible-gaussian-splatting/eval_t4'
print('Tanks & Temples:', TAT_DIR)
print('Deep Blending  :', DB_DIR)
print('Output         :', OUTPUT_DIR)

In [ ]:
# Clone repo & install dependencies (single consolidated cell)
%cd /content
!git clone --recursive https://github.com/DanielCob/accesible-gaussian-splatting.git
%cd /content/accesible-gaussian-splatting

# Core Python deps
!pip install -q plyfile
!pip install -q ./submodules/diff-gaussian-rasterization
!pip install -q ./submodules/simple-knn

# System packages (COLMAP, ImageMagick, xvfb) for conversion & rendering
!apt-get update -y
!apt-get install -y colmap imagemagick xvfb

In [ ]:
# Run evaluation script (training + rendering + metrics at 7k)
# NOTE: We only evaluate Tanks & Temples and Deep Blending (paper comparables)
%cd /content/accesible-gaussian-splatting
!python full_eval.py -tat "{TAT_DIR}" -db "{DB_DIR}" --output_path "{OUTPUT_DIR}"

In [ ]:
# Summarize metrics from results.json files
import os, json, glob
import pandas as pd

rows = []
for scene_dir in sorted(glob.glob(os.path.join(OUTPUT_DIR, '*'))):
    results_path = os.path.join(scene_dir, 'results.json')
    if os.path.isfile(results_path):
        with open(results_path, 'r') as f:
            data = json.load(f)  # {method: {SSIM, PSNR, LPIPS}}
        for method, metrics in data.items():
            rows.append({
                'scene': os.path.basename(scene_dir),
                'method': method,
                'SSIM': metrics.get('SSIM'),
                'PSNR': metrics.get('PSNR'),
                'LPIPS': metrics.get('LPIPS'),
            })
df = pd.DataFrame(rows)
if not df.empty:
    df = df.sort_values(['scene','method']).reset_index(drop=True)
    print(df.to_string(index=False))
    summary_csv = os.path.join(OUTPUT_DIR, 'summary_metrics.csv')
    df.to_csv(summary_csv, index=False)
    print('\nSaved summary to', summary_csv)
else:
    print('No results found under', OUTPUT_DIR)

In [ ]:
# Copy results back to Drive and also create a zip
import shutil, os

drive_out = '/content/drive/MyDrive/ag_splatting_eval/eval_t4'
os.makedirs(os.path.dirname(drive_out), exist_ok=True)
if os.path.exists(drive_out):
    shutil.rmtree(drive_out)
shutil.copytree(OUTPUT_DIR, drive_out)
print('Copied eval results to', drive_out)

zip_path = drive_out + '.zip'
if os.path.exists(zip_path):
    os.remove(zip_path)
shutil.make_archive(drive_out, 'zip', OUTPUT_DIR)
print('Zipped results at', zip_path)